In [ ]:
import pandas as pd
df = pd.read_excel('task_02.xlsx')

df['entry_date'] = pd.to_datetime(df['entry_date'].astype(str), format='%y%m%d')
df = df.sort_values(by=['entry_date', 'entry_time'])

df['pnl'] = df.apply(
    lambda row: (row['exit_price'] - row['entry_price']) * row['qty']
    if row['side'] == 'BUY'
    else (row['entry_price'] - row['exit_price']) * row['qty'],
    axis=1
)
df['pnl'] = df['pnl'].round(2)

# -------------------------------
# Metrics Calculation
# -------------------------------

# 1. Average profit per trade
avg_profit = df['pnl'].mean()

# 2. Top 5 trades with max profit
top_5_profits = df.nlargest(5, 'pnl')

# 3. Cumulative PnL and Drawdown
df['cumulative_pnl'] = df['pnl'].cumsum()
df['cum_max'] = df['cumulative_pnl'].cummax()
df['drawdown'] = df['cumulative_pnl'] - df['cum_max']
df['drawdown_pct'] = (df['drawdown'] / df['cum_max'].replace(0, 1)) * 100

# Max drawdown values
max_drawdown = df['drawdown'].min()
max_drawdown_pct = df['drawdown_pct'].min()

# 4. Win/Loss metrics
total_trades = len(df)
winning_trades = (df['pnl'] > 0).sum()
losing_trades = (df['pnl'] <= 0).sum()
total_profit = df['pnl'].sum()

# -------------------------------
# Save metrics summary
# -------------------------------

summary = pd.DataFrame({
    'Metric': [
        'Average Profit per Trade',
        'Max Drawdown',
        'Max Drawdown (%)',
        'Total Trades',
        'Winning Trades',
        'Losing Trades',
        'Net Profit/Loss'
    ],
    'Value': [
        round(avg_profit, 2),
        round(max_drawdown, 2),
        f"{round(max_drawdown_pct, 2)}%",
        total_trades,
        winning_trades,
        losing_trades,
        round(total_profit, 2)
    ]
})

summary.to_csv('result.csv', index=False)

top_5_profits.to_csv('top_5_profits.csv', index=False)

print("✅ Metrics saved to result.csv")
print("✅ Top 5 profit trades saved to top_5_profits.csv")


✅ Metrics saved to result.csv
✅ Top 5 profit trades saved to top_5_profits.csv
